In [ ]:
import random
import pandas as pd

# Константы
NUM_BUSES = 8
NUM_DRIVERS = 10
NUM_8HR_DRIVERS = 5
NUM_12HR_DRIVERS = NUM_DRIVERS - NUM_8HR_DRIVERS
SHIFT_DURATIONS = {"8hr": 8, "12hr": 12}
DAYS_OF_WEEK = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
TIME_SLOTS = range(24)

# Параметры генетического алгоритма
POPULATION_SIZE = 50
MAX_GENERATIONS = 100
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.8


# Проверка, является ли день отдыха для водителя
def is_rest_day(driver_id, day_idx):
    if driver_id < NUM_8HR_DRIVERS:  # 8-часовые водители отдыхают на выходных
        return DAYS_OF_WEEK[day_idx] in ["Saturday", "Sunday"]
    else:  # 12-часовые водители отдыхают 2 дня после работы
        return day_idx % 3 != 0


# Генерация случайного расписания
def generate_random_schedule():
    schedule = []
    for day_idx, day in enumerate(DAYS_OF_WEEK):
        daily_schedule = []
        for driver_id in range(NUM_DRIVERS):
            if is_rest_day(driver_id, day_idx):
                daily_schedule.append({"driver": driver_id, "bus": None, "start": None, "end": None, "rest": True})
            else:
                bus = random.randint(0, NUM_BUSES - 1)
                shift_type = "8hr" if driver_id < NUM_8HR_DRIVERS else "12hr"
                start_time = random.randint(6, 10) if shift_type == "8hr" else random.randint(0, 23)
                end_time = (start_time + SHIFT_DURATIONS[shift_type]) % 24
                daily_schedule.append({
                    "driver": driver_id,
                    "bus": bus,
                    "start": start_time,
                    "end": end_time,
                    "rest": False
                })
        schedule.append(daily_schedule)
    return schedule


# Вычисление фитнес-функции
def fitness(schedule):
    penalty = 0
    bus_coverage = {day: {hour: [] for hour in TIME_SLOTS} for day in DAYS_OF_WEEK}

    for day_idx, daily_schedule in enumerate(schedule):
        for driver_schedule in daily_schedule:
            if driver_schedule["rest"]:
                continue

            bus = driver_schedule["bus"]
            start, end = driver_schedule["start"], driver_schedule["end"]

            if bus is not None:
                hours = range(start, end + 1 if end >= start else end + 24)
                for hour in hours:
                    bus_coverage[DAYS_OF_WEEK[day_idx]][hour % 24].append(bus)

            if driver_schedule["driver"] < NUM_8HR_DRIVERS and DAYS_OF_WEEK[day_idx] in ["Saturday", "Sunday"]:
                penalty += 10

            if driver_schedule["driver"] >= NUM_12HR_DRIVERS:
                if day_idx > 0:
                    prev_day = schedule[day_idx - 1]
                    for prev_driver in prev_day:
                        if prev_driver["driver"] == driver_schedule["driver"] and not prev_driver["rest"]:
                            penalty += 10

    for day, coverage in bus_coverage.items():
        for hour, buses in coverage.items():
            if len(set(buses)) < NUM_BUSES:
                penalty += 5 * (NUM_BUSES - len(set(buses)))

    return -penalty


# Кроссовер
def crossover(parent1, parent2):
    point = random.randint(0, len(parent1) - 1)
    return parent1[:point] + parent2[point:]


# Мутация
def mutate(schedule):
    day_idx = random.randint(0, len(schedule) - 1)
    driver_idx = random.randint(0, NUM_DRIVERS - 1)
    shift_type = "8hr" if driver_idx < NUM_8HR_DRIVERS else "12hr"
    duration = SHIFT_DURATIONS[shift_type]
    bus = random.randint(0, NUM_BUSES - 1)
    start_time = random.randint(6, 10) if shift_type == "8hr" else random.randint(0, 23)
    end_time = (start_time + duration) % 24
    schedule[day_idx][driver_idx] = {
        "driver": driver_idx,
        "bus": bus,
        "start": start_time,
        "end": end_time,
        "rest": False
    }
    return schedule

# Генетический алгоритм
def genetic_algorithm():
    population = [generate_random_schedule() for _ in range(POPULATION_SIZE)]
    for generation in range(MAX_GENERATIONS):
        population.sort(key=fitness, reverse=True)
        next_generation = population[:5]

        while len(next_generation) < POPULATION_SIZE:
            if random.random() < CROSSOVER_RATE:
                parent1, parent2 = random.sample(population[:20], 2)
                child = crossover(parent1, parent2)
            else:
                child = random.choice(population[:20])

            if random.random() < MUTATION_RATE:
                child = mutate(child)

            next_generation.append(child)

        population = next_generation
        #print(f"Generation {generation + 1}: Best Fitness = {fitness(population[0])}")

    return population[0]


# Форматирование расписания в DataFrame
def generate_schedule_dataframe(schedule):
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    driver_labels = [f"Driver {d + 1}" for d in range(NUM_DRIVERS)]
    shifts = ["8hr" if d < NUM_8HR_DRIVERS else "12hr" for d in range(NUM_DRIVERS)]

    schedule_df = pd.DataFrame(index=driver_labels, columns=["Shift"] + days)
    schedule_df["Shift"] = shifts

    for day_idx, daily_schedule in enumerate(schedule):
        for driver_schedule in daily_schedule:
            driver = f"Driver {driver_schedule['driver'] + 1}"
            if driver_schedule["rest"]:
                schedule_df.loc[driver, days[day_idx]] = "Rest"
            else:
                bus = driver_schedule["bus"]
                start = driver_schedule["start"]
                end = driver_schedule["end"]
                schedule_df.loc[driver, days[day_idx]] = f"Bus {bus + 1} ({start:02d}:00 - {end:02d}:00)"

    return schedule_df.sort_values(by="Shift", ascending=False)


# Запуск
best_schedule = genetic_algorithm()
schedule_df = generate_schedule_dataframe(best_schedule)
print(schedule_df)

          Shift                 Monday                Tuesday  \
Driver 1    8hr  Bus 6 (10:00 - 18:00)  Bus 2 (08:00 - 16:00)   
Driver 2    8hr  Bus 6 (06:00 - 14:00)  Bus 4 (10:00 - 18:00)   
Driver 3    8hr  Bus 2 (10:00 - 18:00)  Bus 5 (10:00 - 18:00)   
Driver 4    8hr  Bus 6 (09:00 - 17:00)  Bus 8 (09:00 - 17:00)   
Driver 5    8hr  Bus 6 (08:00 - 16:00)  Bus 7 (06:00 - 14:00)   
Driver 6   12hr  Bus 3 (19:00 - 07:00)  Bus 2 (04:00 - 16:00)   
Driver 7   12hr  Bus 4 (01:00 - 13:00)  Bus 3 (00:00 - 12:00)   
Driver 8   12hr  Bus 5 (22:00 - 10:00)  Bus 7 (12:00 - 00:00)   
Driver 9   12hr  Bus 8 (11:00 - 23:00)  Bus 8 (06:00 - 18:00)   
Driver 10  12hr  Bus 2 (19:00 - 07:00)  Bus 6 (11:00 - 23:00)   

                       Wednesday               Thursday  \
Driver 1   Bus 8 (10:00 - 18:00)  Bus 6 (10:00 - 18:00)   
Driver 2   Bus 2 (08:00 - 16:00)  Bus 2 (10:00 - 18:00)   
Driver 3   Bus 6 (08:00 - 16:00)  Bus 4 (07:00 - 15:00)   
Driver 4   Bus 6 (09:00 - 17:00)  Bus 6 (08:00 -